# **0 Befor Running:**

If you are running this tutorial on google colab, make sure to set runtime type to gpu, which will accelerate the running speed.

If you are running this tutorial locally, make sure you have already setup python3 + jupyter notebook environments.

# **1 Download Materials**

Materials downloaded includes:



*   Basic source codes (concrete codes please check [my personal github repository](https://github.com/Askfk/maca.git))
*   Required packages
*   Well-trained weights
*   CocoVG dataset


---



Basic source code directory named maca will be download to `"../content/maca/"`

Well-trained weights **maca_cocovg.hdf5** will be downloaded to `"../content/maca/weights/maca_cocovg.hdf5"`

CocoVG dataset will be downloaded and unzipped to `"../content/maca/CocoVG/"`

## **1.1 Download basic source code**

In [ ]:
# Download basic source code, Askfk is the user name of my personal github account.
!git clone https://github.com/Askfk/maca.git

## **1.2 Download and import required packages**
These packages requirements are all wrote in requirements.txt, required packages include **tensorflow == 2.3.0, scipy, numpy, matplotlib, pillow, absl-py, pycocotools, six, keras_applications and skimage**. If there is any packages required in follow codes than cannot be donwloaded successfully here, please download them manully by **!pip install packagename**

In [ ]:
# Install all required packages
!pip install -r /content/maca/requirements.txt

In [ ]:
# import necessary packages and setup project path
import os
import sys
import random
import math
import re
import time
import numpy as np
import json
import tensorflow as tf


print(tf.__version__)

# Initialize the file path
ROOT_DIR = os.path.abspath("/content")
if not os.path.exists(ROOT_DIR):
    os.mkdir(ROOT_DIR)

MACA_DIR = os.path.join(ROOT_DIR, 'maca')
WEIGHT_DIR = os.path.join(MACA_DIR, 'weights')
DATASET_DIR = os.path.join(MACA_DIR, 'COCOVG')

sys.path.append(ROOT_DIR)  # To find local version of the library

from maca.macacripts import utils
from maca.macacripts import visualize
import maca.MACA as MACA
from maca.macacripts.utils import log
from maca.config import Config
from maca.macacripts.cocovg import CocoVGConfig, CocoVGDataset
from maca.pycocoevalcap.eval import eval

MODEL_DIR = os.path.join(ROOT_DIR, "logs")
WEIGHT_PATH = os.path.join(WEIGHT_DIR, "maca_cocovg.hdf5")

## **1.3 Download well-trained weights**

Well-trained weights maca_cocovg.hdf5 will be downloaded to **/content/maca/weights/maca_cocovg.hdf5**

In [ ]:
if not os.path.exists(WEIGHT_DIR):
    os.mkdir(WEIGHT_DIR)
    utils.download_trained_weights(WEIGHT_PATH)

## **1.4 Download CocoVG dataset**

CocoVG dataset zip file will be downloaded to **/content** and then be unzipped to **/content/maca/COCOVG/**

In [ ]:
if not os.path.exists(DATASET_DIR):
    utils.download_dataset("/content/cocovg.zip")
    !unzip -uq '/content/cocovg.zip' -d "/content/maca"

# **2 Initialize materials**


*   Dataset
*   Tokenizer
*   Model
*   Plot model architecture

## **2.1 Initialize dataset**

Initialize train dataset and validation dataset

In [ ]:
dataset_train = CocoVGDataset()
dataset_train.initalize_dataset(DATASET_DIR, 'train')
dataset_train.prepare()

dataset_val = CocoVGDataset()
dataset_val.initalize_dataset(DATASET_DIR, 'val')
dataset_val.prepare()

## **2.2 Initialize Tokenizer**

Initialize tokenizer for further convert sentence to token vector or convert token vector back to sencence.

In [ ]:
with open(os.path.join(DATASET_DIR, 'tokenizer.json'), 'r') as load_f:
  js_tok = json.load(load_f)
tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(js_tok)

## **2.3 Initialize MACA in training mode**

For training MACA, it is necessary to initialize MACA in training mode.

In [ ]:
# Create model in training mode
model = modellib.MACA(mode="training", config=config,
                      model_dir=MODEL_DIR, tokenizer=tokenizer)
model.load_weights(WEIGHT_PATH, by_name=True)

## **2.4 Plot model architecture**
Show MACA model architecture and running flow by table and diagram.

In [ ]:
model.model.summary()

In [ ]:
tf.keras.utils.plot_model(model.model, to_file=os.path.join(ROOT_DIR, 'model_improvement_training.png'), show_shapes=True,
                          show_layer_names=True)

# **Train the model in 3 steps**


*   Training the heads of MACA
*   Training middle (LN, MDN and CDN) parts
*   Training the whole model



# **1. Train heads**

In [ ]:
model.train(dataset_train, dataset_val, 
            learning_rate=0.00001,
            epochs=20,
            layers="heads",
            verbose=1)

# **2. Train middle parts**

In [ ]:
model.train(dataset_train, dataset_val, 
            learning_rate=0.00001,
            epochs=40,
            layers="3+",
            verbose=1)

# **3. Train the whole model**

In [ ]:
model.train(dataset_train, dataset_val, 
            learning_rate=0.000001,
            epochs=80,
            layers="all",
            verbose=1)